In [1]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from brainiak.fcma.util import compute_correlation
from sklearn.metrics import pairwise_distances
import matplotlib.animation as animation
from scipy.stats import spearmanr

In [2]:
# Load the RSA matrices (regions x TRs x subjects x subjects)
npy_file = '/Volumes/ARCHIVES/thesis_pipeline/data/IS_TR_RSA_ROIS/IS_TR_RSA_matrices_all_ROIs.npy'
rsa_matrices = np.load(npy_file)

# Display the shape and some data to ensure it's loaded correctly
print(f"Loaded RSA matrices shape: {rsa_matrices.shape}")
print(f"Sample data for region 0, TR 0:\n{rsa_matrices[0, 0]}")


Loaded RSA matrices shape: (6, 1291, 8, 8)
Sample data for region 0, TR 0:
[[1.00000012e+00 3.58280063e-01 3.63430709e-01 3.70546639e-01
  4.28657144e-01 3.77661496e-01 5.32333314e-01 3.16591710e-01]
 [3.58280063e-01 1.00000000e+00 6.19233131e-01 4.74108160e-01
  3.37872148e-01 3.70413274e-01 4.83037442e-01 1.72313392e-01]
 [3.63430709e-01 6.19233131e-01 1.00000012e+00 3.32206488e-01
  3.04912984e-01 3.76750588e-01 3.73264045e-01 1.67341948e-01]
 [3.70546639e-01 4.74108160e-01 3.32206488e-01 1.00000012e+00
  3.51038009e-01 3.66806298e-01 5.24829090e-01 5.12081504e-01]
 [4.28657144e-01 3.37872148e-01 3.04912984e-01 3.51038009e-01
  1.00000024e+00 3.47161293e-01 3.74337554e-01 3.49258572e-01]
 [3.77661496e-01 3.70413274e-01 3.76750588e-01 3.66806298e-01
  3.47161293e-01 1.00000012e+00 3.81696016e-01 2.19954163e-04]
 [5.32333314e-01 4.83037442e-01 3.73264045e-01 5.24829090e-01
  3.74337554e-01 3.81696016e-01 1.00000000e+00 4.38317716e-01]
 [3.16591710e-01 1.72313392e-01 1.67341948e-01 5.1

In [3]:
# Get the shape details from the loaded data
regions, trs, subjects, _ = rsa_matrices.shape

# Extract a sample subject-by-subject RSA matrix for region 0, TR 0
subject_corr_matrix = rsa_matrices[0, 0, :, :]

# Get the lower triangle (excluding the diagonal)
lower_triangle = np.tril_indices(subjects, -1)
observed_values = subject_corr_matrix[lower_triangle]

# Display the extracted values
print(f"Lower triangle values for region 0, TR 0:\n{observed_values}")


Lower triangle values for region 0, TR 0:
[3.58280063e-01 3.63430709e-01 6.19233131e-01 3.70546639e-01
 4.74108160e-01 3.32206488e-01 4.28657144e-01 3.37872148e-01
 3.04912984e-01 3.51038009e-01 3.77661496e-01 3.70413274e-01
 3.76750588e-01 3.66806298e-01 3.47161293e-01 5.32333314e-01
 4.83037442e-01 3.73264045e-01 5.24829090e-01 3.74337554e-01
 3.81696016e-01 3.16591710e-01 1.72313392e-01 1.67341948e-01
 5.12081504e-01 3.49258572e-01 2.19954163e-04 4.38317716e-01]


In [5]:
# Compute the mean of the lower triangle for the observed values
observed_mean_corr = np.mean(observed_values)

# Display the observed mean correlation
print(f"Observed mean correlation for region 0, TR 0: {observed_mean_corr}")


Observed mean correlation for region 0, TR 0: 0.37159645284607123


In [6]:
# Shuffle the subject order (permute rows and columns)
permuted_indices = np.random.permutation(subjects)
permuted_matrix = subject_corr_matrix[permuted_indices, :][:, permuted_indices]

# Extract the lower triangle for the permuted matrix
permuted_values = permuted_matrix[lower_triangle]

# Display the permuted lower triangle values
print(f"Permuted lower triangle values:\n{permuted_values}")


Permuted lower triangle values:
[6.19233131e-01 1.67341948e-01 1.72313392e-01 3.76750588e-01
 3.70413274e-01 2.19954163e-04 3.32206488e-01 4.74108160e-01
 5.12081504e-01 3.66806298e-01 3.04912984e-01 3.37872148e-01
 3.49258572e-01 3.47161293e-01 3.51038009e-01 3.63430709e-01
 3.58280063e-01 3.16591710e-01 3.77661496e-01 3.70546639e-01
 4.28657144e-01 3.73264045e-01 4.83037442e-01 4.38317716e-01
 3.81696016e-01 5.24829090e-01 3.74337554e-01 5.32333314e-01]


In [7]:
# Compute the mean of the lower triangle for the permuted values
permuted_mean_corr = np.mean(permuted_values)

# Display the permuted mean correlation
print(f"Permuted mean correlation for region 0, TR 0: {permuted_mean_corr}")


Permuted mean correlation for region 0, TR 0: 0.37159645284607123


In [8]:
n_permutations = 10
null_distribution = np.zeros(n_permutations)

for perm in range(n_permutations):
    # Shuffle the subject order
    permuted_indices = np.random.permutation(subjects)
    permuted_matrix = subject_corr_matrix[permuted_indices, :][:, permuted_indices]
    
    # Extract the lower triangle for the permuted matrix
    permuted_values = permuted_matrix[lower_triangle]
    
    # Compute the mean for the permuted lower triangle
    null_distribution[perm] = np.mean(permuted_values)

# Compare the observed mean to the null distribution
print(f"Null distribution:\n{null_distribution}")
print(f"Proportion of permuted correlations >= observed: {np.mean(null_distribution >= observed_mean_corr)}")


Null distribution:
[0.37159645 0.37159645 0.37159645 0.37159645 0.37159645 0.37159645
 0.37159645 0.37159645 0.37159645 0.37159645]
Proportion of permuted correlations >= observed: 1.0
